In [1]:
!pip install torchattacks
!pip install pandas
!pip install openpyxl
!pip install numpy
!pip install plot 
!pip install numpy


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import torch
import torch.nn as nn
import time
import pandas as pd

In [3]:
import Sources.DefaultModel as Util
import Sources.MnistModel as MnistModel
import Sources.TestUtils as TestUtils

In [4]:
Device = Util.HardwareIdentification()
TrainDataLoaderMNIST,TestDataLoaderMNIST = MnistModel.ImportData(1)

cuda


100%|██████████| 9912422/9912422 [00:16<00:00, 605340.80it/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 4771751.90it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



100%|██████████| 1648877/1648877 [00:02<00:00, 611071.76it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw

60000
10000


In [ ]:
Model = MnistModel.ConvNeuralNet(10)
Model = Model.to(Device)
Epochs = 40
Loss = nn.CrossEntropyLoss()
LearningRate = 0.001
Optimizer = torch.optim.Adam(Model.parameters(), lr=LearningRate)

In [ ]:
%%script false --no-raise-error
for Epoch in range(Epochs):
    ReturnLoss = Util.Trainloop(TrainDataLoaderMNIST, Model, Loss, Optimizer, Device)
    print('{:.4f}'.format(ReturnLoss))
ModelPath = 'Models\\ModelMNIST.torch'
torch.save(Model.state_dict(), ModelPath)

In [ ]:
Model.load_state_dict(torch.load('Models\\ModelMNIST.torch'))

In [ ]:
Util.TestLoop(TestDataLoaderMNIST, Model, Device)
X, y = next(iter(TestDataLoaderMNIST))

In [ ]:
ActivationList = [3, 4]
CoefList = [5]
CList = [10, 100]
ProbList = [4]
SimilarityList = [0.2]
Init = 0
lr = 0.2
Steps = 10000

data = []
for S in SimilarityList:
  for P in ProbList:
    for C in CList:
      for Coef in CoefList:
        for Activation in ActivationList:
          DataResult, _ = TestUtils.AttackNorms(Model, C, Steps, lr,  Coef, Activation, P, S, Init, X, y, Device)
          data += DataResult

df = pd.DataFrame(data, columns=['L0 Norm', 'L2 Norm', 'Linf Norm', 'Activation', 'Coef', 'Prob', 'c', 'Similarity', 'Pre', 'Ppred', 'Apred', 'Spre'])
df.to_excel("Results\\Parameters\\Parameters.xlsx")

In [ ]:
Activation = 4
Coef = 5
Prob = 2
Similarity = 0.2
Init=1
lr = 0.2
Steps = 10000


data = []
start_time = time.time()
data = TestUtils.TestLoopC(Model, Steps, lr, Coef, Activation, Prob, Similarity, Init, X, y, Device, 1000, 0.1, 1.5)
end_time = time.time()

TestUtils.SaveTime(start_time, end_time, "Results\\MNIST\\ComputationTimeResults.txt")
TestUtils.SaveData(data, "Results\\MNIST\\Results.xlsx")

In [ ]:
Coef=5
Activation=4
Prob = 2
Similarity = 0.2
Init=0
lr = 0.2

TestedIterations = [1000, 10000]

for i in range((len(TestedIterations))):
    data = []
    start_time = time.time()
    data = TestUtils.TestLoopC(Model, TestedIterations[i], lr, Coef, Activation, Prob, Similarity, Init, X, y, Device, 1000, 0.1, 1.5)
    end_time = time.time()

    TestUtils.SaveTime(start_time, end_time, "Results\\MNIST\\TimeXM" + str(TestedIterations[i]) +".txt")
    TestUtils.SaveData(data, "Results\\MNIST\\ResultXM" + str(TestedIterations[i]) + ".xlsx")
    

In [ ]:
Coef=5
Activation=4
Prob = 2
Similarity = 0.2
Init=0
lr = 0.2
Steps = 10000

TestUtils.SaveLoopC(Model, Steps, lr, Coef, Activation, Prob, Similarity, Init, X, y, Device, 1000, 0.1, 1.5, "Results\\MNIST\\Images\\")

In [ ]:
dataJSMA = []

start_time = time.time()
Data = TestUtils.TestJSMA(1,1, Model, X, y, Device)
end_time = time.time()

TestUtils.SaveTime(start_time, end_time, "Results\\MNIST\\TimeJSMAM.txt")
TestUtils.SaveData(Data, "Results\\MNIST\\BestJSMAM.xlsx")

In [ ]:
Count=0
dataCW = []

start_time = time.time()
Data, AX = TestUtils.TestCW(100, 1, 10, Model, X, y, Device)
end_time = time.time()

TestUtils.SaveTime(start_time, end_time, "Results\\MNIST\\TimeCWM.txt")
TestUtils.SaveData([Data], "Results\\MNIST\\BestCWM.xlsx")